## Week 5 inclass
#### Group 1: Tara Bode, Hankun Li

Part A: Text-based industry classification
Cluster the business sections extracted in assignment 3 into 50 clusters (using the cosine similarity) to group firms with similar business descriptions together.

Submit your code and the final output csv file, with CIK, conformed end of period, and the cluster number for each business section file.

Note: it is likely you will need to tokenize/word count each business section file before reading the next one. (If you try to read all business sections before doing any further processing you will probably run out of memory).

In [4]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

You need to simply take the.txt files from HiperGator and use the path (r'/blue/acg7849/share/BS/') 

so we can grab the length of each file for the csv file.

Also, we need to use the for loop to cluster the business section files and mimic the class code from yesterday to grab the top 20 common words.

csv file, with CIK, conformed end of period, and file name (business section file) and the cluster number for each business section file.

 # recommended - use summary files - got initial csv

In [5]:
# matching filenames

with open ('/blue/acg7849/share/BS/summary.txt') as a: 
    summary = a. readlines()
print (summary[0:5])

sumdf = pd.DataFrame (summary)
#sumdf.str.split(pat="|")

new = sumdf[0].str.split("|", expand = True)
sumdf["CIK"]= new[0]
sumdf["coName"]= new[1]
sumdf["formtype"]= new[2]
sumdf["date"]= new[3]
sumdf["fName"]= new[4]
sumdf["length"]= new[5]
sumdf = sumdf.drop(columns=[0])
sumdf = sumdf.drop([0])

print (sumdf[0:15])                        
#Initialize csv - no clusters
#sumdf.to_csv('ASG4.csv', index = True)

['CIK|coname|formtype|date|filename|length\n', '0001289850|NeuroMetrix, Inc.|10-K|20171231|267762.txt|132616\n', '0001345016|YELP INC|10-K|20191231|280603.txt|152323\n', '0001245791|GI DYNAMICS, INC.|10-K|20171231|267435.txt|28236\n', '0001772177|KURA SUSHI USA, INC.|10-K|20190831|275323.txt|38468\n']
           CIK                                             coName formtype  \
1   0001289850                                  NeuroMetrix, Inc.     10-K   
2   0001345016                                           YELP INC     10-K   
3   0001245791                                  GI DYNAMICS, INC.     10-K   
4   0001772177                               KURA SUSHI USA, INC.     10-K   
5   0000832428                                    E.W. SCRIPPS Co     10-K   
6   0001513761                Norwegian Cruise Line Holdings Ltd.     10-K   
7   0001678361  World Omni Automobile Lease Securitization Tru...     10-K   
8   0001029199                              EURONET WORLDWIDE INC     10-

In [6]:
sumdf["length"]= pd.to_numeric(sumdf["length"])
print (sumdf[0:15])

           CIK                                             coName formtype  \
1   0001289850                                  NeuroMetrix, Inc.     10-K   
2   0001345016                                           YELP INC     10-K   
3   0001245791                                  GI DYNAMICS, INC.     10-K   
4   0001772177                               KURA SUSHI USA, INC.     10-K   
5   0000832428                                    E.W. SCRIPPS Co     10-K   
6   0001513761                Norwegian Cruise Line Holdings Ltd.     10-K   
7   0001678361  World Omni Automobile Lease Securitization Tru...     10-K   
8   0001029199                              EURONET WORLDWIDE INC     10-K   
9   0001341766                             Celsius Holdings, Inc.     10-K   
10  0001679826                        Ping Identity Holding Corp.     10-K   
11  0001549631                                   Quarta-Rad, Inc.     10-K   
12  0001077771                               UMPQUA HOLDINGS COR

In [7]:
# filter out all the things with <1000 length 
sumdf = sumdf.loc[sumdf["length"] >= 1000]
print (sumdf[0:15])     # seems filtering success! 

sumdf_bk  = sumdf  # make another file as backup
sumdf_test  = sumdf[:100]  # test length = 100

           CIK                               coName formtype      date  \
1   0001289850                    NeuroMetrix, Inc.     10-K  20171231   
2   0001345016                             YELP INC     10-K  20191231   
3   0001245791                    GI DYNAMICS, INC.     10-K  20171231   
4   0001772177                 KURA SUSHI USA, INC.     10-K  20190831   
5   0000832428                      E.W. SCRIPPS Co     10-K  20181231   
6   0001513761  Norwegian Cruise Line Holdings Ltd.     10-K  20171231   
8   0001029199                EURONET WORLDWIDE INC     10-K  20171231   
9   0001341766               Celsius Holdings, Inc.     10-K  20191231   
10  0001679826          Ping Identity Holding Corp.     10-K  20191231   
13  0001650445                   Quorum Health Corp     10-K  20191231   
14  0000049600             EASTGROUP PROPERTIES INC     10-K  20191231   
15  0001464300        ALL MARKETING SOLUTIONS, INC.     10-K  20171231   
17  0001174891                       C

# proceed for clustering

In [8]:
newdirectory = '/blue/acg7849/share/BS/item1'
#file_list1 = glob.glob(newdirectory + '/*.txt')
#print(file_list1) # success!

In [9]:
sumdf["full_link"]= newdirectory +'/'+ sumdf["fName"]
print (sumdf[0:13]) 

           CIK                               coName formtype      date  \
1   0001289850                    NeuroMetrix, Inc.     10-K  20171231   
2   0001345016                             YELP INC     10-K  20191231   
3   0001245791                    GI DYNAMICS, INC.     10-K  20171231   
4   0001772177                 KURA SUSHI USA, INC.     10-K  20190831   
5   0000832428                      E.W. SCRIPPS Co     10-K  20181231   
6   0001513761  Norwegian Cruise Line Holdings Ltd.     10-K  20171231   
8   0001029199                EURONET WORLDWIDE INC     10-K  20171231   
9   0001341766               Celsius Holdings, Inc.     10-K  20191231   
10  0001679826          Ping Identity Holding Corp.     10-K  20191231   
13  0001650445                   Quorum Health Corp     10-K  20191231   
14  0000049600             EASTGROUP PROPERTIES INC     10-K  20191231   
15  0001464300        ALL MARKETING SOLUTIONS, INC.     10-K  20171231   
17  0001174891                       C

In [10]:
listoflink = list(sumdf["full_link"]) # in case we need a list
print (listoflink[0:5])

['/blue/acg7849/share/BS/item1/267762.txt', '/blue/acg7849/share/BS/item1/280603.txt', '/blue/acg7849/share/BS/item1/267435.txt', '/blue/acg7849/share/BS/item1/275323.txt', '/blue/acg7849/share/BS/item1/277539.txt']


# list of files (raw text)

In [11]:
allFiles_raw = [] 

# process first 10 files
for i in listoflink[0:100]:  # will crash if do all at once
    with open( i, encoding='utf-8') as f:
        content = f.read()
        
    # append file to allFiles
    allFiles_raw.append(content ) # plain text

In [12]:
print(len(allFiles_raw))

100


In [13]:
print(type(allFiles_raw))
print(type(allFiles_raw[0]))

<class 'list'>
<class 'str'>


# piazza advices - avoid HPG crash (skip)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [30]:
documents = (
"The sky is blue",
"The sun is bright",
"The sun in sky is bright",
"We can see the shining sun, bright sun"
)

In [31]:
#tfidf = vectorizer.fit_transform(generator) # should be a txt object/tuple/list of txt?
tfidf_test = vectorizer.fit_transform(documents)
tfidff = vectorizer.fit_transform(allFiles_raw)
print (tfidff)
#print (allFiles_raw[1])

  (0, 43)	0.003647863062960956
  (0, 19987)	0.0034155099368387036
  (0, 20222)	0.0025281901344859025
  (0, 129)	0.002760543260608156
  (0, 13446)	0.0013133869940178285
  (0, 511)	0.0020027279340754083
  (0, 15838)	0.0032352828504582393
  (0, 9345)	0.003975346401076231
  (0, 17090)	0.003975346401076231
  (0, 10452)	0.003975346401076231
  (0, 5906)	0.0009759934298421682
  (0, 17408)	0.0012556714974309082
  (0, 7099)	0.001640870332133102
  (0, 19525)	0.00229583700836365
  (0, 13336)	0.0016636231970336927
  (0, 19350)	0.002200706796370629
  (0, 17735)	0.001535829894079695
  (0, 11732)	0.001968353670248376
  (0, 8864)	0.0017881265838679118
  (0, 17736)	0.0030182896197770885
  (0, 9331)	0.0012282901196303852
  (0, 2045)	0.002246872120565892
  (0, 55)	0.002157038145850191
  (0, 1681)	0.001535829894079695
  (0, 4289)	0.0020386304354268214
  :	:
  (99, 8979)	0.03335804470549861
  (99, 20049)	0.18853304183802358
  (99, 18184)	0.2570215639617925
  (99, 6214)	0.057482224480335846
  (99, 12018)	0.0

In [33]:
def docGen():
    yield documents[0]
    yield documents[1]
    yield documents[2]
    yield documents[3]

SyntaxError: invalid character in identifier (<ipython-input-33-1e80a839aa1e>, line 2)

In [35]:
def docGen():
    yield documents[0]
    yield documents[1]
    yield documents[2]
    yield documents[3]


SyntaxError: invalid character in identifier (<ipython-input-35-1e80a839aa1e>, line 2)

In [34]:
tfidf = vectorizer.fit_transform(docGen() )
print ('size tfidf matrix (#documents, #unique words):', tfidf.shape)

print('unique words', vectorizer.get_feature_names() )

print(tfidf)

NameError: name 'docGen' is not defined

In [ ]:
# generator
class MyCommentLetters(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        #for fname in os.listdir(self.dirname)[0:2]:
        for fname in os.listdir(self.dirname)[0:200]:
            
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                # cleaner (but slower)
                yield [ x for x in word_tokenize(line) if x.isalpha() and x.lower() not in stopWords and x not in punc ]
                # faster (but not so clean)
                #yield line.lower().split()
                
import gensim
# min-count is how minimum count of each word, default is 5
# workers: #cores (may need cython to be installed for this to have an effect)
model = gensim.models.Word2Vec(wordLists, min_count=2, workers=8)

# kmeans - try with raw files list!

In [14]:
import numpy as np
import nltk
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [15]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [16]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [17]:
totalvocab_stemmed = []
totalvocab_tokenized = []

file_list1 = listoflink 
#print(file_list1) # success!

# allFiles_raw is list of string/files
# this avoids to read in and combine a list of strings 

In [18]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in allFiles_raw:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'filing1', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [19]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 648158 items in vocab_frame


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(allFiles_raw) #fit the vectorizer to a iterable list of txt

print(tfidf_matrix.shape)

/home/hli1/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CPU times: user 15.4 s, sys: 122 ms, total: 15.5 s
Wall time: 15.6 s
(100, 1376)


In [21]:
terms = tfidf_vectorizer.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [22]:
from sklearn.cluster import KMeans

num_clusters = 50                            # doc num > num of clst

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.17 s


In [23]:
import joblib
#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle
joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [24]:
print(clusters) # success up to now, list of clulsters can be appended to csv

[12, 9, 12, 11, 9, 48, 49, 1, 0, 20, 10, 37, 15, 14, 8, 12, 35, 2, 22, 1, 4, 4, 7, 0, 17, 30, 5, 41, 28, 12, 10, 46, 8, 21, 16, 40, 17, 14, 44, 20, 20, 3, 33, 18, 20, 5, 24, 15, 2, 46, 47, 14, 13, 0, 25, 27, 34, 32, 18, 3, 34, 0, 24, 17, 14, 29, 46, 40, 24, 4, 6, 11, 30, 49, 13, 38, 20, 19, 45, 4, 44, 4, 31, 28, 43, 21, 42, 36, 0, 23, 17, 4, 39, 9, 8, 3, 23, 23, 26, 19]


# Appending clusters to df!

In [28]:
# based on the 100-file version

sumdf_test["cluster"]= clusters
print (sumdf_test[0:10])  # test success
sumdf_test.to_csv('ASG4-test.csv', index = True)

           CIK                               coName formtype      date  \
1   0001289850                    NeuroMetrix, Inc.     10-K  20171231   
2   0001345016                             YELP INC     10-K  20191231   
3   0001245791                    GI DYNAMICS, INC.     10-K  20171231   
4   0001772177                 KURA SUSHI USA, INC.     10-K  20190831   
5   0000832428                      E.W. SCRIPPS Co     10-K  20181231   
6   0001513761  Norwegian Cruise Line Holdings Ltd.     10-K  20171231   
8   0001029199                EURONET WORLDWIDE INC     10-K  20171231   
9   0001341766               Celsius Holdings, Inc.     10-K  20191231   
10  0001679826          Ping Identity Holding Corp.     10-K  20191231   
13  0001650445                   Quorum Health Corp     10-K  20191231   

         fName  length  cluster  
1   267762.txt  132616       12  
2   280603.txt  152323        9  
3   267435.txt   28236       12  
4   275323.txt   38468       11  
5   277539.txt 

<ipython-input-28-03c3c01d41ba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumdf_test["cluster"]= clusters
